In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as sp

from time import time

In [2]:
sites = ['site%s' % i for i in range(1, 11)]
df_train = pd.read_csv('train_sessions.csv', dtype={s: 'str' for s in sites})

for s in sites:
    df_train[s].fillna('na', inplace=1)

In [3]:
df = df_train

In [4]:
df['sites'] = df.site1 + ' ' + df.site2 + ' ' + df.site3 + ' ' + df.site4 + ' ' + df.site5 + ' ' + \
              df.site6 + ' ' + df.site7 + ' ' + df.site8 + ' ' + df.site9 + ' ' + df.site10  

In [20]:
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import train_test_split

In [26]:
cv = CountVectorizer(token_pattern='\S+')#, min_df=10, max_df=0.5)
X_ohe = cv.fit_transform(df_train.sites)

In [27]:
y = df_train.target.values

In [28]:
n = len(df_train) // 10

In [32]:
X = X_ohe

In [29]:
X_train = X[:-n]
y_train = y[:-n]

X_val = X[-n:]
y_val = y[-n:]

In [30]:
for C in [0.01, 0.1, 0.5, 1]:
    t0 = time()

    svm = LogisticRegression(penalty='l1', dual=False, C=C, random_state=1)
    svm.fit(X_train, y_train)

    y_pred = svm.decision_function(X_val)
    auc = roc_auc_score(y_val, y_pred)

    print('C=%s, took %.3fs, auc=%.3f' % (C, time() - t0, auc))

C=0.01, took 0.717s, auc=0.860
C=0.1, took 0.991s, auc=0.935
C=0.5, took 2.189s, auc=0.955
C=1, took 4.438s, auc=0.961


In [33]:
C = 1
svm = LogisticRegression(penalty='l1', dual=False, C=C, random_state=1)
svm.fit(X, y)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=1, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [34]:
df_test = pd.read_csv('test_sessions.csv', dtype={s: 'str' for s in sites})

for s in sites:
    df_test[s].fillna('na', inplace=1)

In [36]:
df = df_test
df['sites'] = df.site1 + ' ' + df.site2 + ' ' + df.site3 + ' ' + df.site4 + ' ' + df.site5 + ' ' + \
              df.site6 + ' ' + df.site7 + ' ' + df.site8 + ' ' + df.site9 + ' ' + df.site10  

In [37]:
X_test = cv.transform(df_test.sites)


In [40]:
pred = svm.decision_function(X_test)
df_res = pd.DataFrame()
df_res['session_id'] = df_test.session_id
df_res['target'] = pred

In [41]:
df_res.to_csv('benchmark7.csv', index=False)

In [42]:
!gzip benchmark7.csv